# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb36816f9d0>
├── 'a' --> tensor([[ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991]])
└── 'x' --> <FastTreeValue 0x7fb36816ffd0>
    └── 'c' --> tensor([[ 0.5127,  0.8892,  0.0086, -0.4732],
                        [-0.7450, -2.1816, -0.8962,  0.6720],
                        [ 0.9687,  0.0585, -0.0568, -2.0104]])

In [4]:
t.a

tensor([[ 2.6605, -0.3240, -0.0506],
        [ 0.7118,  1.5301,  0.5991]])

In [5]:
%timeit t.a

75.7 ns ± 0.0674 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb36816f9d0>
├── 'a' --> tensor([[ 0.3870, -1.5840,  0.2773],
│                   [ 0.6147, -0.9161,  0.7920]])
└── 'x' --> <FastTreeValue 0x7fb36816ffd0>
    └── 'c' --> tensor([[ 0.5127,  0.8892,  0.0086, -0.4732],
                        [-0.7450, -2.1816, -0.8962,  0.6720],
                        [ 0.9687,  0.0585, -0.0568, -2.0104]])

In [7]:
%timeit t.a = new_value

75.7 ns ± 0.0813 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991]]),
    x: Batch(
           c: tensor([[ 0.5127,  0.8892,  0.0086, -0.4732],
                      [-0.7450, -2.1816, -0.8962,  0.6720],
                      [ 0.9687,  0.0585, -0.0568, -2.0104]]),
       ),
)

In [10]:
b.a

tensor([[ 2.6605, -0.3240, -0.0506],
        [ 0.7118,  1.5301,  0.5991]])

In [11]:
%timeit b.a

66.3 ns ± 0.0762 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2534, -0.0822, -0.5458],
               [ 0.1031, -0.7378,  0.5325]]),
    x: Batch(
           c: tensor([[ 0.5127,  0.8892,  0.0086, -0.4732],
                      [-0.7450, -2.1816, -0.8962,  0.6720],
                      [ 0.9687,  0.0585, -0.0568, -2.0104]]),
       ),
)

In [13]:
%timeit b.a = new_value

552 ns ± 4.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

902 ns ± 1.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

150 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

150 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb2892267c0>
├── 'a' --> tensor([[[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]],
│           
│                   [[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]],
│           
│                   [[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]],
│           
│                   [[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]],
│           
│                   [[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]],
│           
│                   [[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]],
│           
│                   [[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]],
│           
│                   [[ 2.6605, -0.3240, -0.0506],
│                    [ 0.7118,  1.5301,  0.5991]]])
└── 'x' --> <FastTreeValue 0x7fb289226100>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.5 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb36815d970>
├── 'a' --> tensor([[ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991],
│                   [ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991],
│                   [ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991],
│                   [ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991],
│                   [ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991],
│                   [ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991],
│                   [ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991],
│                   [ 2.6605, -0.3240, -0.0506],
│                   [ 0.7118,  1.5301,  0.5991]])
└── 'x' --> <FastTreeValue 0x7fb2915e6cd0>
    └── 'c' --> tensor([[ 0.5127,  0.8892,  0.0086, -0.4732],
                        [-0.7450, -2.1816, -0.8962,  0.6720],
                 

In [23]:
%timeit t_cat(trees)

31.2 µs ± 42.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

63.8 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]],
       
               [[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]],
       
               [[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]],
       
               [[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]],
       
               [[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]],
       
               [[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]],
       
               [[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]],
       
               [[ 2.6605, -0.3240, -0.0506],
                [ 0.7118,  1.5301,  0.5991]]]),
    x: Batch(
           c: tensor([[[ 0.5127,  0.8892,  0.0086, -0.4732],
                       [-0.7450, -2.1816, -0.8962,  0.6720],
                       [ 0.9687,  0.0585, -0.0568, -2.0104]],
         

In [26]:
%timeit Batch.stack(batches)

84.4 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991],
               [ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991],
               [ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991],
               [ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991],
               [ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991],
               [ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991],
               [ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991],
               [ 2.6605, -0.3240, -0.0506],
               [ 0.7118,  1.5301,  0.5991]]),
    x: Batch(
           c: tensor([[ 0.5127,  0.8892,  0.0086, -0.4732],
                      [-0.7450, -2.1816, -0.8962,  0.6720],
                      [ 0.9687,  0.0585, -0.0568, -2.0104],
                      [ 0.5127,  0.8892,  0.0086, -0.4732],
                      [-0.7450, -

In [28]:
%timeit Batch.cat(batches)

153 µs ± 440 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

383 µs ± 7.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
